In [150]:
%matplotlib inline
import os
import pandas as pd
import deepdish as dp
import numpy as np
import re
import fnmatch

# libs for plotting
import bokeh
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.models.tools import HoverTool
from bokeh.models import LinearColorMapper, ColorBar, BasicTicker, Select, PrintfTickFormatter
from bokeh.transform import jitter
from bokeh.sampledata.commits import data
from bokeh.palettes import brewer
import holoviews
from bokeh.models import SingleIntervalTicker, LinearAxis, FixedTicker
# libs for stats
import time
from sklearn.manifold import TSNE

# print in notebok
output_notebook()

Loading BokehJS ...

In [6]:
# data paths for uvvis data
datapath = '/Volumes/extreme/repos/lab-data/characterization/uvvisnir/'
datapath_generic = '*generic*'
datapath_fluids  = '_fluids'
datapath_exclude = ['_special-studies','_summaries','CO2-00001']
files_exclude    = ['.*','_*','transmission*', '*.xlsm']

In [221]:
# make a dataframe with the files
ff = []

excludes = r'|'.join([fnmatch.translate(x) for x in files_exclude]) or r'$.'
for root, dirs, files in os.walk(datapath):
    [dirs.remove(d) for d in list(dirs) if d in datapath_exclude]
    files = [f for f in files if not re.match(excludes, f)]
    #[files.remove(f) for f in list(files) if f in ['.','_spectralon','.xlsm']]
    for name in files:
        filename = os.path.join(root,name)
        flag_generic = 0
        flag_fluid   = 0
        subsample_tag = '0'
        name = name.replace('.csv','')
        name = name.replace('.Sample','')
        name = name.replace('_sec','')
        name = name.replace('_','-')
        name = name.replace('generic','00000')
        if re.match('(?s:.*generic.*)', filename):
            flag_generic = 1
        if re.match('(?s:.*fluid.*)',filename):
            flag_fluid = 1

        # check if it has some subsample info
        sample_tag    = '-'.join(name.split('-')[0:2])
        if len(name.split('-'))>2:
            subsample_tag = '-'.join(name.split('-')[2::])
        mat_type = name.split('-')[0]
        ff.append([filename, sample_tag, subsample_tag, mat_type, flag_generic, flag_fluid])
        #ff = ff.append([filename, sample_tag, subsample_tag, flag_generic, flag_fluid])
        print(name, sample_tag, subsample_tag, flag_generic, flag_fluid, sep='\t')
df = pd.DataFrame(ff, columns=['filename','sample','sub-sample','mat_type','generic','fluid'])

AH-00001-1	AH-00001	1	0	0
AH-00001-2	AH-00001	2	0	0
SH-00003-B	SH-00003	B	0	0
SH-00003-T	SH-00003	T	0	0
SH-00003	SH-00003	0	0	0
SH-00004-2A-B	SH-00004	2A-B	0	0
SH-00004-2A-F	SH-00004	2A-F	0	0
SH-00004-2A-T	SH-00004	2A-T	0	0
SH-00004-3A-B	SH-00004	3A-B	0	0
SH-00004-3A-T	SH-00004	3A-T	0	0
SH-00004-4A-B	SH-00004	4A-B	0	0
SH-00004-4A-T	SH-00004	4A-T	0	0
SH-00004-5A-B	SH-00004	5A-B	0	0
SH-00004-5A-T	SH-00004	5A-T	0	0
SH-00006-B	SH-00006	B	0	0
SH-00006-T	SH-00006	T	0	0
SH-00007-B	SH-00007	B	0	0
SH-00007-T	SH-00007	T	0	0
SH-00008-B	SH-00008	B	0	0
SH-00008-T	SH-00008	T	0	0
H2O+a-00001	H2O+a-00001	0	0	1
H2O+g-00001-01	H2O+g-00001	01	0	1
H2O+g-00001-02	H2O+g-00001	02	0	1
D2O-00000	D2O-00000	0	1	1
H2O-00000	H2O-00000	0	1	1
SiOil-00000	SiOil-00000	0	1	1
air-00000	air-00000	0	1	1
CB-00000	CB-00000	0	1	0
CR-00000-1	CR-00000	1	1	0
CR-00000-2	CR-00000	2	1	0
CR-00000-3	CR-00000	3	1	0
DLe-00000	DLe-00000	0	1	0
LSi-00000	LSi-00000	0	1	0
LSp-00000	LSp-00000	0	1	0
LSsy-00000-1	LSsy-00000	1	1	0
LSsy-00000-2

In [222]:
df_list = []
for i in range(0,df.shape[0]):
    dftemp = pd.read_csv(df.loc[i,'filename'], skiprows=2, names=['lambda','alpha'])
    dftemp = dftemp.loc[(dftemp['lambda']>=900) & (dftemp['lambda']<=2500),:].loc[::-1,:].reset_index(drop=True).copy()
    dftemp = dftemp.transpose().copy()
    dftemp.columns = dftemp.loc['lambda',:].values
    dftemp.drop('lambda',axis=0, inplace=True)
    dftemp.reset_index(drop=True, inplace=True)
    df_list.append(dftemp)
dfs = pd.concat(df_list, ignore_index=True)
data = pd.concat([df.iloc[:,1:],dfs], sort=False, axis=1)
data.dropna(axis=1, inplace=True)
del df_list, dftemp, df, dfs

In [223]:
# save the data under summaries
dp.io.save(os.path.join(datapath,'_summaries','compiled-11202018.h5'),data)

/sandbox/opt/share/anaconda/lib/python3.6/site-packages/deepdish/io/hdf5io.py:581: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->axis0] [items->None]

  filters=filters, idtable=idtable)


In [237]:
data['TOC'] = 0
data.loc[data['mat_type']=='SH','TOC'] = data.loc[data['mat_type']=='SH',:].iloc[:,5::].apply(np.average, axis=1).values

In [238]:

#data = dp.io.load(os.path.join(datapath,'_summaries','compiled-11202018.h5'),data)

,sample,sub-sample,mat_type,generic,fluid,900.0,902.0,904.0,906.0,908.0,...,1184.0,1186.0,1188.0,1190.0,1192.0,1194.0,1196.0,1198.0,1200.0,TOC
0,AH-00001,1,AH,0,0,1.0110,1.0106,1.0120,1.0122,1.0122,...,0.9944,0.9938,0.9929,0.9930,0.9920,0.9905,0.9924,0.9929,0.9919,0.000000
1,AH-00001,2,AH,0,0,0.7478,0.7481,0.7490,0.7487,0.7497,...,0.7821,0.7822,0.7822,0.7824,0.7824,0.7815,0.7835,0.7837,0.7838,0.000000
2,SH-00003,B,SH,0,0,1.0803,1.0814,1.0796,1.0836,1.0800,...,1.0675,1.0672,1.0679,1.0663,1.0666,1.0648,1.0644,1.0655,1.0666,1.066366
3,SH-00003,T,SH,0,0,1.3440,1.3411,1.3322,1.3378,1.3394,...,1.3206,1.3212,1.3230,1.3190,1.3203,1.3219,1.3180,1.3181,1.3206,1.320249
4,SH-00003,0,SH,0,0,1.1016,1.0988,1.0983,1.0970,1.0972,...,1.0869,1.0879,1.0857,1.0859,1.0844,1.0855,1.0866,1.0876,1.0857,1.084983


In [ ]:
#data.index = data['sample']+ '-' + data['sub-sample'].astype('str')

ds = data.loc[data['fluid']==0,:]
ds_cats = ds['sample'] + '-' + ds['sub-sample'].astype('str')

def vstac_spectra(data, data_cats, x='x', y='y'):
    # prepare the data
    
    ds.set_index(ds_cats, inplace=True)
    ds.index.name='material'
    ds.columns.astype('str')
    ds = ds.iloc[:,5:].copy()
    ds.columns.name = 'lambda'

In [66]:
#data.fillna(0, inplace=True)

In [79]:
ds = data.loc[data['fluid']==0,:]
ds_cats = ds['sample'] + '-' + ds['sub-sample'].astype('str')
ds.set_index(ds_cats, inplace=True)
ds.index.name='material'
ds.columns.astype('str')
ds = ds.iloc[:,5::1].copy()
ds.columns.name = 'lambda'
#ds.loc[:,'sub-sample'] = ds.loc[:,'sub-sample'].apply(lambda x: str(x).replace('None',''))
#ds_cats = ds['sample'] + '-' + ds['sub-sample'].astype('str').replace('None','')

In [239]:
data

,sample,sub-sample,mat_type,generic,fluid,900.0,902.0,904.0,906.0,908.0,...,1184.0,1186.0,1188.0,1190.0,1192.0,1194.0,1196.0,1198.0,1200.0,TOC
0,AH-00001,1,AH,0,0,1.0110,1.0106,1.0120,1.0122,1.0122,...,0.9944,0.9938,0.9929,0.9930,0.9920,0.9905,0.9924,0.9929,0.9919,0.000000
1,AH-00001,2,AH,0,0,0.7478,0.7481,0.7490,0.7487,0.7497,...,0.7821,0.7822,0.7822,0.7824,0.7824,0.7815,0.7835,0.7837,0.7838,0.000000
2,SH-00003,B,SH,0,0,1.0803,1.0814,1.0796,1.0836,1.0800,...,1.0675,1.0672,1.0679,1.0663,1.0666,1.0648,1.0644,1.0655,1.0666,1.066366
3,SH-00003,T,SH,0,0,1.3440,1.3411,1.3322,1.3378,1.3394,...,1.3206,1.3212,1.3230,1.3190,1.3203,1.3219,1.3180,1.3181,1.3206,1.320249
4,SH-00003,0,SH,0,0,1.1016,1.0988,1.0983,1.0970,1.0972,...,1.0869,1.0879,1.0857,1.0859,1.0844,1.0855,1.0866,1.0876,1.0857,1.084983
5,SH-00004,2A-B,SH,0,0,1.0499,1.0475,1.0458,1.0465,1.0469,...,0.9534,0.9527,0.9516,0.9512,0.9501,0.9500,0.9490,0.9488,0.9502,0.988234
6,SH-00004,2A-F,SH,0,0,0.9607,0.9596,0.9574,0.9590,0.9564,...,0.8902,0.8886,0.8888,0.8881,0.8882,0.8875,0.8890,0.8876,0.8865,0.914541
7,SH-00004,2A-T,SH,0,0,0.8518,0.8487,0.8471,0.8473,0.8454,...,0.7384,0.7387,0.7375,0.7371,0.7371,0.7365,0.7350,0.7348,0.7350,0.780422
8,SH-00004,3A-B,SH,0,0,1.0262,1.0246,1.0212,1.0224,1.0232,...,0.9321,0.9322,0.9310,0.9288,0.9299,0.9289,0.9278,0.9284,0.9290,0.967187
9,SH-00004,3A-T,SH,0,0,1.0543,1.0532,1.0507,1.0527,1.0539,...,1.0016,1.0006,1.0006,0.9985,0.9991,0.9988,0.9983,0.9977,1.0003,1.018753


In [ ]:


# plot = bp.figure(plot_width=800, plot_height=200, x_axis_type=None)
ticker = SingleIntervalTicker(interval=5, num_minor_ticks=10)
xaxis = LinearAxis(ticker=ticker)
plot.add_layout(xaxis, 'below')

In [199]:
dss = ds.loc[:,ds.columns>=1000.0].loc[:,::2].copy()
df = pd.DataFrame(dss.stack(), columns=['rate']).reset_index()
df['lambdas'] = df['lambda'].astype('str')

lambdas   = df['lambdas'].unique()
materials = df['material'].unique()

mapper = LinearColorMapper(palette='Viridis256', 
                           low=df['rate'].min(), high=df['rate'].max())

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = figure(title='Rock absorbtion',
           x_range=lambdas, y_range=list(reversed(materials)), x_axis_type=None, 
           x_axis_location="above", plot_width=960, plot_height=1000,
           tools=TOOLS, toolbar_location='below',
           tooltips=[('lambda', '@lambdas'), ('I', '@rate%'), ('rock','@material')])

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "10pt"
p.axis.major_label_standoff = 2
p.xaxis.major_label_orientation = 'vertical'
#p.xaxis.ticker = ['1000','1500','2000']#np.linspace(1100, 2400, 11)
p.xaxis.visible = True
p.xaxis.major_label_overrides ={1: 'A', 200: 'B', 300: 'C'}

p.rect(x="lambdas", y="material", width=3, height=0.9,
       source=df,
       fill_color={'field': 'rate', 'transform': mapper},
       line_color=None)

color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="10pt",
                     ticker=BasicTicker(desired_num_ticks=5),
                     label_standoff=6, border_line_color=None, location=(0, 0),
                     title = 'absorbtion (a.u.)', title_text_align="left")
p.add_layout(color_bar, 'right')

ticker=FixedTicker(ticks=[0,42.67,366.67, 500, 1000])
# #ticker = SingleIntervalTicker(interval=100, num_minor_ticks=10)
xaxis = LinearAxis(ticker=ticker)
xaxis.major_label_overrides = {0: '1000 nm', 42.67: '1064 nm', 366.67: '1550 nm' ,500: '1750 nm', 1000: '2500 nm'}
xaxis.major_label_orientation = 'vertical'
p.add_layout(xaxis, 'above')

show(p)

In [ ]:
dss = ds.loc[:,ds.columns>=1000.0].loc[:,::2].copy()
df = pd.DataFrame(dss.stack(), columns=['rate']).reset_index()
df['lambdas'] = df['lambda'].astype('str')

lambdas   = df['lambdas'].unique()
materials = df['material'].unique()

mapper = LinearColorMapper(palette='Viridis256', 
                           low=df['rate'].min(), high=df['rate'].max())

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = figure(title='Rock absorbtion',
           x_range=lambdas, y_range=list(reversed(materials)), x_axis_type=None, 
           x_axis_location="above", plot_width=960, plot_height=1000,
           tools=TOOLS, toolbar_location='below',
           tooltips=[('lambda', '@lambdas'), ('I', '@rate%'), ('rock','@material')])

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "10pt"
p.axis.major_label_standoff = 2
p.xaxis.major_label_orientation = 'vertical'
#p.xaxis.ticker = ['1000','1500','2000']#np.linspace(1100, 2400, 11)
p.xaxis.visible = True
p.xaxis.major_label_overrides ={1: 'A', 200: 'B', 300: 'C'}

p.rect(x="lambdas", y="material", width=3, height=0.9,
       source=df,
       fill_color={'field': 'rate', 'transform': mapper},
       line_color=None)

color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="10pt",
                     ticker=BasicTicker(desired_num_ticks=5),
                     label_standoff=6, border_line_color=None, location=(0, 0),
                     title = 'absorbtion (a.u.)', title_text_align="left")
p.add_layout(color_bar, 'right')

ticker=FixedTicker(ticks=[0,42.67,366.67, 500, 1000])
# #ticker = SingleIntervalTicker(interval=100, num_minor_ticks=10)
xaxis = LinearAxis(ticker=ticker)
xaxis.major_label_overrides = {0: '1000 nm', 42.67: '1064 nm', 366.67: '1550 nm' ,500: '1750 nm', 1000: '2500 nm'}
xaxis.major_label_orientation = 'vertical'
p.add_layout(xaxis, 'above')

show(p)

In [212]:
re.match('SH',materials)

TypeError: cannot use a string pattern on a bytes-like object